In [ ]:
!pip -q install -U \
  numpy==2.0.2 \
  pandas==2.2.2 \
  scikit-learn==1.6.1 \
  scipy==1.16.3 \
  gensim==4.4.0 \
  sentence-transformers==5.2.3 \
  torch==2.10.0

In [ ]:
# ============================================
# Full Text Clustering Pipeline
# - Features: BOW, TF-IDF, LDA, Embeddings
# - Algorithms: KMeans, EM(GMM), Hierarchical
# - Evaluation: Silhouette, Cohen Kappa (with optimal mapping)
# - Visualization: Bar plots + confusion matrix heatmaps for ALL models
# ============================================

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.preprocessing import LabelEncoder

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

from sklearn.metrics import (
    silhouette_score,
    confusion_matrix,
    cohen_kappa_score
)

from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt

# For word embeddings (Sentence-BERT)
from sentence_transformers import SentenceTransformer

# A bit nicer default plotting (optional)
plt.rcParams["figure.figsize"] = (6, 4)
plt.rcParams["font.size"] = 9


# ============================================
# 1. Load data
# ============================================

DATA_PATH = "processed_data.csv"
TEXT_COLUMN = "text_clean"
LABEL_COLUMN = "category"

df = pd.read_csv(DATA_PATH)

print("Data shape:", df.shape)
print("\nCategory counts:")
print(df[LABEL_COLUMN].value_counts())

texts = df[TEXT_COLUMN].astype(str).tolist()
true_labels_str = df[LABEL_COLUMN].astype(str).tolist()

# Encode string labels (e.g. Colon, Liver...) -> 0..4
le = LabelEncoder()
y_true = le.fit_transform(true_labels_str)

print("\nLabel mapping (index -> class name):")
for i, c in enumerate(le.classes_):
    print(f"{i} -> {c}")

n_clusters = len(le.classes_)
print("\nNumber of clusters (n_clusters):", n_clusters)


# ============================================
# 2. Feature Engineering
#    - BOW + SVD
#    - TF-IDF + SVD
#    - LDA topic distribution (on BOW)
#    - Sentence-BERT embeddings (+ optional SVD)
# ============================================

max_features = 3000
svd_dim = 50
lda_topics = 20

# ----- 2.1 BOW -----
bow_vectorizer = CountVectorizer(max_features=max_features)
X_bow = bow_vectorizer.fit_transform(texts)          # sparse (n_samples, V)

svd_bow = TruncatedSVD(n_components=svd_dim, random_state=42)
X_bow_emb = svd_bow.fit_transform(X_bow)            # dense (n_samples, svd_dim)

print("\n[BOW] raw shape:", X_bow.shape, "embedded shape:", X_bow_emb.shape)

# ----- 2.2 TF-IDF -----
tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
X_tfidf = tfidf_vectorizer.fit_transform(texts)

svd_tfidf = TruncatedSVD(n_components=svd_dim, random_state=42)
X_tfidf_emb = svd_tfidf.fit_transform(X_tfidf)

print("[TF-IDF] raw shape:", X_tfidf.shape, "embedded shape:", X_tfidf_emb.shape)

# ----- 2.3 LDA topics (using BOW as input) -----
lda = LatentDirichletAllocation(
    n_components=lda_topics,
    random_state=42,
    learning_method="batch"
)
X_lda = lda.fit_transform(X_bow)    # (n_samples, n_topics)
print("[LDA] topic distribution shape:", X_lda.shape)

# ----- 2.4 Sentence-BERT embeddings -----
print("\n[Embeddings] Encoding with Sentence-BERT (this may take a bit) ...")
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
X_embed_raw = embed_model.encode(texts, show_progress_bar=True)  # (n_samples, d)

print("[Embeddings] raw embedding shape:", X_embed_raw.shape)

# Optional: reduce embedding dim with SVD for fair comparison
svd_emb = TruncatedSVD(n_components=svd_dim, random_state=42)
X_emb = svd_emb.fit_transform(X_embed_raw)
print("[Embeddings] reduced embedding shape:", X_emb.shape)


# ============================================
# 3. Cohen Kappa with optimal cluster→label mapping
# ============================================

def compute_kappa_with_optimal_mapping(y_true, cluster_labels):
    """
    Steps:
      1) Compute confusion matrix: rows = true labels, cols = cluster IDs
      2) Use Hungarian algorithm to find best cluster -> true label mapping
      3) Remap cluster_labels to pseudo-labels aligned with y_true
      4) Compute Cohen Kappa on (y_true, mapped_labels)
    Returns:
      kappa, mapping, cm_raw, cm_mapped, mapped_labels
    """
    cm_raw = confusion_matrix(y_true, cluster_labels)

    # We want to maximize diagonal -> minimize -cm
    cost_matrix = -cm_raw.T  # (n_clusters x n_classes)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    mapping = {cluster_id: true_label_idx
               for cluster_id, true_label_idx in zip(row_ind, col_ind)}

    mapped_labels = np.vectorize(mapping.get)(cluster_labels)
    kappa = cohen_kappa_score(y_true, mapped_labels)
    cm_mapped = confusion_matrix(y_true, mapped_labels)

    return kappa, mapping, cm_raw, cm_mapped, mapped_labels


def evaluate_clustering(model_name, feature_name, y_true, X_emb, cluster_labels):
    """
    Compute:
      - Silhouette
      - Cohen Kappa (after optimal mapping)
      - Confusion matrix (mapped)
    Return a result dict with everything we need later.
    """
    sil = silhouette_score(X_emb, cluster_labels)
    kappa, mapping, cm_raw, cm_mapped, mapped_labels = compute_kappa_with_optimal_mapping(
        y_true, cluster_labels
    )

    full_name = f"{model_name} ({feature_name})"
    print(f"\n=== {full_name} ===")
    print("Silhouette score:", sil)
    print("Cohen Kappa:", kappa)
    print("Cluster -> True index mapping:", mapping)
    print("True index -> class name:")
    for i, c in enumerate(le.classes_):
        print(f"  {i} -> {c}")
    print("\nConfusion matrix AFTER mapping (rows=true, cols=pred):")
    print(cm_mapped)

    result = {
        "Model": model_name,
        "Feature": feature_name,
        "Name": full_name,
        "Silhouette": sil,
        "Kappa": kappa,
        "Mapping": mapping,
        "CM_raw": cm_raw,
        "CM_mapped": cm_mapped,
        "cluster_labels": cluster_labels,
        "mapped_labels": mapped_labels,
    }
    return result


# ============================================
# 4. Define clustering runners
# ============================================

def run_kmeans(X, n_clusters, random_state=42):
    km = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)
    return km.fit_predict(X)

def run_gmm(X, n_clusters, random_state=42):
    gmm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gmm.fit(X)
    return gmm.predict(X)

def run_agg(X, n_clusters):
    agg = AgglomerativeClustering(n_clusters=n_clusters, linkage="ward")
    return agg.fit_predict(X)


# ============================================
# 5. Run clustering on all feature types + all algorithms
# ============================================

features = {
    "BOW": X_bow_emb,
    "TF-IDF": X_tfidf_emb,
    "LDA": X_lda,
    "EMB": X_emb,
}

algorithms = {
    "KMeans": run_kmeans,
    "EM_GMM": run_gmm,
    "Hierarchical": run_agg,
}

results = []

for feat_name, X_feat in features.items():
    print("\n" + "="*30)
    print(f"Clustering with feature: {feat_name}")
    print("="*30)
    for algo_name, algo_func in algorithms.items():
        labels = algo_func(X_feat, n_clusters)
        res = evaluate_clustering(algo_name, feat_name, y_true, X_feat, labels)
        results.append(res)


# ============================================
# 6. Summarize metrics + bar plots
# ============================================

metrics_df = pd.DataFrame(
    [{
        "Name": r["Name"],
        "Model": r["Model"],
        "Feature": r["Feature"],
        "Silhouette": r["Silhouette"],
        "Kappa": r["Kappa"],
    } for r in results]
)

print("\n=== Summary metrics (all models) ===")
print(metrics_df)

# --- Silhouette bar plot ---
plt.figure()
plt.bar(metrics_df["Name"], metrics_df["Silhouette"])
plt.xticks(rotation=60, ha="right")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Scores by Model & Feature")
plt.tight_layout()
plt.show()

# --- Kappa bar plot ---
plt.figure()
plt.bar(metrics_df["Name"], metrics_df["Kappa"])
plt.xticks(rotation=60, ha="right")
plt.ylabel("Cohen Kappa")
plt.title("Cohen Kappa by Model & Feature")
plt.tight_layout()
plt.show()


# ============================================
# 7. Confusion matrix heatmaps for ALL models
# ============================================

def plot_confusion_matrix(cm, title, class_names):
    plt.figure()
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45, ha="right")
    plt.yticks(tick_marks, class_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.show()

class_names = list(le.classes_)

for r in results:
    title = f"Confusion Matrix (mapped) - {r['Name']}"
    cm = r["CM_mapped"]
    plot_confusion_matrix(cm, title, class_names)


# ============================================
# 8. Pick champion model (by Kappa)
# ============================================

champ_idx = metrics_df["Kappa"].idxmax()
champ_row = metrics_df.loc[champ_idx]
champ_name = champ_row["Name"]

print("\n===============================")
print("Champion model (by Kappa):")
print(champ_row)
print("===============================")


In [ ]:
# ==========================================================
# ERROR ANALYSIS PIPELINE FOR CHAMPION CLUSTERING MODEL
# ==========================================================
# This module performs structured post-hoc error analysis
# for the selected champion model (by highest Kappa score).
# It includes:
#   1. Champion model selection
#   2. Overall error computation
#   3. Class-wise error distribution
#   4. Confusion structure analysis
#   5. Visualization of misclassification patterns
#   6. Cluster-level keyword interpretation
#   7. Attractor cluster investigation
#   8. Representative error examples
# ==========================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# ----------------------------------------------------------
# 1. Utility Function: Retrieve Model Results by Name
# ----------------------------------------------------------
def get_result_by_name(results, name):
    for r in results:
        if r["Name"] == name:
            return r
    raise ValueError(f"Result with Name='{name}' not found.")


# ----------------------------------------------------------
# 2. Utility Function: Extract Top TF-IDF Words for Cluster
# ----------------------------------------------------------
def top_words_from_tfidf_cluster(X_tfidf_raw, vocab, mask, topk=15):
    mean_vec = X_tfidf_raw[mask].mean(axis=0).A1
    top_idx = mean_vec.argsort()[::-1][:topk]
    return vocab[top_idx], mean_vec[top_idx]


# ----------------------------------------------------------
# 3. Main Procedure: Structured Error Analysis
# ----------------------------------------------------------
def run_error_analysis_auto(df, TEXT_COLUMN, LABEL_COLUMN_STR, le, y_true,
                            results, metrics_df, X_tfidf_raw, tfidf_vectorizer,
                            topk_words=15, n_examples=5):

    # ------------------------------------------------------
    # 3.1 Champion Model Selection (Highest Kappa)
    # ------------------------------------------------------
    champ_idx = metrics_df["Kappa"].idxmax()
    champ_row = metrics_df.loc[champ_idx]
    champ_name = champ_row["Name"]
    champ_res = get_result_by_name(results, champ_name)

    print("\n===============================")
    print("Champion model (by Kappa):")
    print(champ_row)
    print("===============================\n")

    # ------------------------------------------------------
    # 3.2 Construct Prediction and Error DataFrame
    # ------------------------------------------------------
    y_pred_mapped = champ_res["mapped_labels"]
    cluster_labels = champ_res["cluster_labels"]
    mapping = champ_res["Mapping"]  # cluster_id -> true_idx
    class_names = list(le.classes_)

    df_err = df.copy()
    df_err["true_idx"] = y_true
    df_err["pred_idx"] = y_pred_mapped
    df_err["true_name"] = [class_names[i] for i in y_true]
    df_err["pred_name"] = [class_names[i] for i in y_pred_mapped]
    df_err["cluster_id"] = cluster_labels
    df_err["is_error"] = (df_err["true_idx"] != df_err["pred_idx"])

    # ------------------------------------------------------
    # 3.3 Overall Error Rate Computation
    # ------------------------------------------------------
    err_rate = df_err["is_error"].mean()
    err_count = int(df_err["is_error"].sum())
    print(f"[B1] Overall Error Rate: {err_rate:.3f}  ( {err_count} / {len(df_err)} )\n")

    # ------------------------------------------------------
    # 3.4 Class-wise Error Rate Distribution
    # ------------------------------------------------------
    err_by_class = df_err.groupby("true_name")["is_error"].mean().sort_values(ascending=False)
    print("[B2] Error Rate by True Class:")
    print(err_by_class, "\n")

    # ------------------------------------------------------
    # 3.5 Confusion Matrix Construction
    # ------------------------------------------------------
    conf_table = pd.crosstab(df_err["true_name"], df_err["pred_name"])
    print("[B3] Confusion Table (rows=true, cols=pred):")
    print(conf_table, "\n")

    # ------------------------------------------------------
    # 3.6 Top Misclassification Pairs (Error Only)
    # ------------------------------------------------------
    pairs = (df_err[df_err["is_error"]]
             .groupby(["true_name", "pred_name"])
             .size()
             .sort_values(ascending=False))
    print("[C] Top Confusion Pairs (Errors Only):")
    print(pairs.head(10), "\n")

    # ------------------------------------------------------
    # 3.7 Visualization: Class-wise Error Rate
    # ------------------------------------------------------
    plt.figure(figsize=(6,4))
    plt.bar(err_by_class.index, err_by_class.values)
    plt.ylabel("Error Rate")
    plt.title(f"Error Rate by True Class\nChampion: {champ_name}")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

    # ------------------------------------------------------
    # 3.8 Visualization: Misclassification Flow (Stacked Bar)
    # ------------------------------------------------------
    err_only = df_err[df_err["is_error"]]
    if len(err_only) > 0:
        mis_dist = pd.crosstab(err_only["true_name"], err_only["pred_name"])
        mis_dist.plot(kind="bar", stacked=True, figsize=(7,4))
        plt.ylabel("Number of Misclassified Documents")
        plt.title(f"Misclassification Destinations\nChampion: {champ_name}")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()

    # ------------------------------------------------------
    # 3.9 Cluster-Level Keyword Analysis (TF-IDF)
    # ------------------------------------------------------
    vocab = np.array(tfidf_vectorizer.get_feature_names_out())

    print("\n[Cluster Keyword Analysis] Top TF-IDF Words per Mapped Class:")

    for true_idx, true_name in enumerate(class_names):
        cluster_id = [cid for cid, tidx in mapping.items() if tidx == true_idx][0]
        mask = (cluster_labels == cluster_id)
        top_words, _ = top_words_from_tfidf_cluster(
            X_tfidf_raw, vocab, mask, topk=topk_words
        )
        print(f"\n- Class: {true_name}  |  cluster_id: {cluster_id}")
        print("  Top words:", list(top_words))

    # ------------------------------------------------------
    # 3.10 Attractor Cluster Identification
    # ------------------------------------------------------
    if len(err_only) > 0:
        attractor = err_only["pred_name"].value_counts().idxmax()
        attractor_idx = class_names.index(attractor)
        attractor_cluster = [cid for cid, tidx in mapping.items() if tidx == attractor_idx][0]

        print("\n[Attractor Analysis]")
        print("Predicted Label Receiving Most Errors:", attractor)
        print("Corresponding cluster_id:", attractor_cluster)

        mask_attr = (cluster_labels == attractor_cluster)
        top_words_attr, _ = top_words_from_tfidf_cluster(
            X_tfidf_raw, vocab, mask_attr, topk=topk_words
        )
        print("Top words in Attractor Cluster:", list(top_words_attr))

    # ------------------------------------------------------
    # 3.11 Representative Error Examples
    # ------------------------------------------------------
    if len(pairs) > 0:
        true_p, pred_p = pairs.index[0]
        print(f"\n[Representative Examples] Most Common Confusion:")
        print(f"TRUE={true_p}  ->  PRED={pred_p}")

        ex = df_err[(df_err["true_name"]==true_p) &
                    (df_err["pred_name"]==pred_p)].head(n_examples)

        for _, row in ex.iterrows():
            print("\n--- Example ---")
            print("TRUE:", row["true_name"],
                  "PRED:", row["pred_name"],
                  "cluster_id:", row["cluster_id"])
            snippet = str(row[TEXT_COLUMN])[:600]
            print("TEXT (clean snippet):", snippet,
                  "..." if len(snippet)==600 else "")

    # ------------------------------------------------------
    # 3.12 Return Analysis Outputs
    # ------------------------------------------------------
    return {
        "champ_row": champ_row,
        "champ_res": champ_res,
        "df_err": df_err,
        "err_by_class": err_by_class,
        "conf_table": conf_table,
        "pairs": pairs
    }


# ----------------------------------------------------------
# 4. Execute Error Analysis
# ----------------------------------------------------------
analysis_out = run_error_analysis_auto(
    df=df,
    TEXT_COLUMN=TEXT_COLUMN,
    LABEL_COLUMN_STR=LABEL_COLUMN,
    le=le,
    y_true=y_true,
    results=results,
    metrics_df=metrics_df,
    X_tfidf_raw=X_tfidf,
    tfidf_vectorizer=tfidf_vectorizer,
    topk_words=15,
    n_examples=5
)